# Demo show_spots_napari

This notebook walks through using Napari to view starfish data.

In [1]:
%gui qt5

import matplotlib.pyplot as plt
import numpy as np
import os

from starfish import data, FieldOfView
from starfish.types import Indices
from starfish.image import Filter

from starfish.imagestack.imagestack import ImageStack
from starfish.intensity_table.intensity_table import IntensityTable

## Load and view data
First, we load the Allen Institute smFISH data. We can view the raw data by calling the ImageStack.show_stack_napari() method. We can provide indices (in the same style as get_slice() to slice the ImageStack.

In [2]:
use_test_data = os.getenv("USE_TEST_DATA") is not None
experiment = data.allen_smFISH(use_test_data=use_test_data)
primary_image = experiment.fov()[FieldOfView.PRIMARY_IMAGES]

100%|██████████| 102/102 [07:40<00:00,  5.67s/it]


In [4]:
viewer, ind = primary_image.show_stack_napari({})

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## Visualize detected spots
Next, we perform the image processing pipeline (from the Allen Institute smFISH demo notebook). To save time, we load the decoded and detected spots from disk. We can then view the spots overlaid on the 3D ImageStack using the plot.show_spots_napari() method. 

In [3]:
# Filter the image

clip = Filter.Clip(p_min=10, p_max=100)
clip.run(primary_image, verbose=True, in_place=True)

bandpass = Filter.Bandpass(lshort=0.5, llong=7, truncate=4)
bandpass.run(primary_image, verbose=True, in_place=True)

clip = Filter.Clip(p_min=10, p_max=100, is_volume=False)
clip.run(primary_image, verbose=True, in_place=True)

sigma=(1, 0, 0)  # filter only in z, do nothing in x, y
glp = Filter.GaussianLowPass(sigma=sigma, is_volume=True)
glp.run(primary_image, in_place=True, verbose=True)

102it [00:00, 156.95it/s]
102it [00:00, 131.20it/s]
3it [00:00, 12.05it/s]


<starfish.ImageStack (r: 1, c: 3, z: 34, y: 2048, x: 2048)>

In [3]:
# Load the decoded data
decoded = IntensityTable.load('allen_decoded')

# Displaying spots on a 3D image volume
show_spots_napari() projects the provided background image (mirroring language from decoded spots - maybe we should rename) along the rnd and channel axes and overlays the spots as ring markers.

Spot marker parameters (e.g., opacity, size, color) can be adjusted by double clicking the "markers" box in the right panel. They can also be toggled on/off by clicking the "eye" checkbox icon.

In [6]:
import starfish.plot

starfish.plot.show_spots_napari(spots=decoded, background_image=primary_image)

100%|██████████| 34/34 [00:00<00:00, 105.84it/s]


# Z projected images
show_spots_napari can also take images projected along the z axis and automatically project the decoded spots along the z axis.


In [5]:
mp = primary_image.max_proj(Indices.CH, Indices.ROUND, Indices.Z)

starfish.plot.show_spots_napari(spots=decoded, background_image=mp)

100%|██████████| 1/1 [00:00<00:00, 128.56it/s]
